In [17]:
import pandas as pd
import numpy as np
import sys
import csv

In [18]:
XML_results = pd.read_csv("XML_CLEAN_INPUT_082019.csv") 

In [19]:
list(XML_results.columns.values)

['Alias',
 'Software_Package',
 'Identifier',
 'Pub_Year',
 'DOI',
 'Journal_Title',
 'Article_ID',
 'File_Name',
 'Parent1_Tag',
 'Parent2_Tag',
 'Parent3_Tag',
 'Parent4_Tag',
 'Parent1_Content',
 'Parent2_Content',
 'Parent3_Content',
 'Author(s)',
 'Publisher',
 'Title']

In [20]:
XML_results.head(5)

,Alias,Software_Package,Identifier,Pub_Year,DOI,Journal_Title,Article_ID,File_Name,Parent1_Tag,Parent2_Tag,Parent3_Tag,Parent4_Tag,Parent1_Content,Parent2_Content,Parent3_Content,Author(s),Publisher,Title
0,astroblend,AstroBlend,0,2016,10.3847/0004-637X/818/2/115,The Astrophysical Journal,"apj521773, 10.3847/0004-637X/818/2/115, 521773...",apj_818_2_115.xml,sc,p,sec,sec,['astroblend'],['We note that our example scripts only explor...,['\n<label>3.5.</label>\n<title>From <sans-ser...,Vogt Frédéric P. A. Owen Chris I. Verdes-Mon...,The American Astronomical Society,ADVANCED DATA VISUALIZATION IN ASTROPHYSICS: T...
1,astroblend.com,AstroBlend,0,2016,10.3847/0004-637X/818/2/115,The Astrophysical Journal,"apj521773, 10.3847/0004-637X/818/2/115, 521773...",apj_818_2_115.xml,ext-link,p,fn,p,['http://www.astroblend.com'],"['\n<ext-link ext-link-type=""uri"" xlink:href=""...",['\n<label><sup>26</sup></label>\n<p>\n<ext-li...,Vogt Frédéric P. A. Owen Chris I. Verdes-Mon...,The American Astronomical Society,ADVANCED DATA VISUALIZATION IN ASTROPHYSICS: T...
2,10.1051/0004-6361/201322068,Astropy,1,2014,10.1088/0004-6256/148/1/13,The Astronomical Journal,"aj493368, ANJOAA, 10.1088/0004-6256/148/1/13, ...",aj_148_1_13.xml,ext-link,nlm-citation,ref,ref-list,['10.1051/0004-6361/201322068'],"['\n<person-group person-group-type=""author"">\...","['\n<nlm-citation citation-type=""journal"">\n<p...",Rodney Steven A.Riess Adam G.Strolger Louis-Gr...,The American Astronomical Society,TYPE Ia SUPERNOVA RATE MEASUREMENTS TO REDSHIF...
3,10.1051/0004-6361/201322068,Astropy,1,2014,10.1088/0004-6256/148/1/14,The Astronomical Journal,"aj495229, ANJOAA, 10.1088/0004-6256/148/1/14, ...",aj_148_1_14.xml,ext-link,nlm-citation,ref,ref-list,['10.1051/0004-6361/201322068'],"['\n<person-group person-group-type=""author"">\...","['\n<nlm-citation citation-type=""journal"">\n<p...",Bañados E.Venemans B. P.Morganson E.Decarli R....,The American Astronomical Society,DISCOVERY OF EIGHT z ∼ 6 QUASARS FROM Pan-STARRS1
4,10.1051/0004-6361/201322068,Astropy,1,2014,10.1088/0004-6256/148/3/53,The Astronomical Journal,"aj499538, ANJOAA, 10.1088/0004-6256/148/3/53, ...",aj_148_3_53.xml,ext-link,nlm-citation,ref,ref-list,['10.1051/0004-6361/201322068'],"['\n<person-group person-group-type=""author"">\...","['\n<nlm-citation citation-type=""journal"">\n<p...",Gullikson KevinDodson-Robinson SarahKraus Adam...,The American Astronomical Society,"CORRECTING FOR TELLURIC ABSORPTION: METHODS, C..."


In [21]:
# Convert tag, content, identifier, pub_year cols to strings

XML_results['Parent1_Tag'] = XML_results['Parent1_Tag'].astype('|S')
XML_results['Parent2_Tag'] = XML_results['Parent2_Tag'].astype('|S')
XML_results['Parent3_Tag'] = XML_results['Parent3_Tag'].astype('|S')
XML_results['Parent4_Tag'] = XML_results['Parent4_Tag'].astype('|S')

XML_results['Parent1_Content'] = XML_results['Parent1_Content'].astype('|S')
XML_results['Parent2_Content'] = XML_results['Parent2_Content'].astype('|S')
XML_results['Parent3_Content'] = XML_results['Parent3_Content'].astype('|S')

XML_results['Identifier'] = XML_results['Identifier'].astype('|S')
XML_results['Pub_Year'] = XML_results['Pub_Year'].astype('|S')
XML_results['Alias'] = XML_results['Alias'].astype('|S')

In [22]:
# Create column for reference section

# If the tag lable or tag content contain any of the following reference elements it will be marked "yes":
# element-citation 
# mixed-citation
# nlm-citation
# ref
# citation-alternatives
# ref-list 
# xref
# source
# bibr
# collab
# contrib-group
# person-group
# pub-id

references = ['Parent1_Tag','Parent2_Tag','Parent3_Tag','Parent4_Tag','Parent1_Content','Parent2_Content']
XML_results["ref"] = np.where((XML_results[references] == "element-citation").any(axis=1) | (XML_results[references]== "mixed-citation").any(axis=1) | (XML_results[references]== "nlm-citation").any(axis=1) | (XML_results[references]== "ref").any(axis=1) | (XML_results[references]== "citation-alternatives").any(axis=1) | (XML_results[references]== "ref-list").any(axis=1) | (XML_results[references]== "xref").any(axis=1) | (XML_results[references]== "source").any(axis=1) | (XML_results[references]== "collab").any(axis=1) | (XML_results[references]== "contrib-group").any(axis=1) | (XML_results[references]== "person-group").any(axis=1) | (XML_results[references]== "pub-id").any(axis=1) | (XML_results[references]== "bibr").any(axis=1), "yes", "no")

In [23]:
# Create column for acknowledgements
# If the tag lable or tag content contains "ack" it will be marked "yes"

acknowledgements = ['Parent1_Tag','Parent2_Tag','Parent3_Tag','Parent4_Tag','Parent1_Content','Parent2_Content']
XML_results["ack"] = np.where((XML_results[acknowledgements] == "ack").any(axis=1), "yes", "no")

In [24]:
# Create column for footnotes
# If the tag lable or tag content contains "fn" or "fn-group" it will be marked "yes"

footnotes = ['Parent1_Tag','Parent2_Tag','Parent3_Tag','Parent4_Tag','Parent1_Content','Parent2_Content']
XML_results["fn"] = np.where((XML_results[footnotes] == "fn").any(axis=1) | (XML_results[footnotes]== "fn-group").any(axis=1), "yes", "no")

In [25]:
# Create column for attempt at recognizable credit (ref + ack + fn + ext-link + back)

# element-citation 
# mixed-citation
# nlm-citation
# ref
# citation-alternatives
# ref-list 
# xref
# source
# bibr
# collab
# contrib-group
# person-group
# pub-id
# ext-link
# back

rec_credit = ['Parent1_Tag','Parent2_Tag','Parent3_Tag','Parent4_Tag','Parent1_Content','Parent2_Content']
XML_results["rec_credit"] = np.where((XML_results[rec_credit] == "element-citation").any(axis=1) | (XML_results[rec_credit]== "mixed-citation").any(axis=1) | (XML_results[rec_credit]== "nlm-citation").any(axis=1) | (XML_results[rec_credit]== "ref").any(axis=1) | (XML_results[rec_credit]== "citation-alternatives").any(axis=1) | (XML_results[rec_credit]== "ref-list").any(axis=1) | (XML_results[rec_credit]== "xref").any(axis=1) | (XML_results[rec_credit]== "source").any(axis=1) | (XML_results[rec_credit]== "collab").any(axis=1) | (XML_results[rec_credit]== "contrib-group").any(axis=1) | (XML_results[rec_credit]== "person-group").any(axis=1) | (XML_results[rec_credit]== "pub-id").any(axis=1) | (XML_results[rec_credit] == "fn").any(axis=1) | (XML_results[rec_credit]== "fn-group").any(axis=1) | (XML_results[rec_credit]== "ack").any(axis=1) |(XML_results[rec_credit]== "back").any(axis=1) |(XML_results[rec_credit]== "ext-link").any(axis=1) | (XML_results[rec_credit]== "bibr").any(axis=1), "yes", "no")

In [26]:
# Check new cols

list(XML_results.columns.values)

['Alias',
 'Software_Package',
 'Identifier',
 'Pub_Year',
 'DOI',
 'Journal_Title',
 'Article_ID',
 'File_Name',
 'Parent1_Tag',
 'Parent2_Tag',
 'Parent3_Tag',
 'Parent4_Tag',
 'Parent1_Content',
 'Parent2_Content',
 'Parent3_Content',
 'Author(s)',
 'Publisher',
 'Title',
 'ref',
 'ack',
 'fn',
 'rec_credit']

In [27]:
XML_results.to_csv("XML_REF_ANALYSIS_082019.csv")

In [28]:
# Proportion of articles with aliases in refrence sections

XML_all_refs = XML_results.loc[XML_results['ref'] == "yes"]
XML_ref_proportion = XML_all_refs.groupby('Software_Package')['File_Name'].nunique()/XML_results.groupby('Software_Package')['File_Name'].nunique()
# XML_ref_proportion.to_csv("XML_ref_proportion_082019.csv")
XML_ref_proportion

Software_Package
AstroBlend           NaN
Astropy         0.860595
RADMC-3D        0.682243
SAOImage DS9    0.167155
Spec2d          0.467105
Stingray        0.500000
TARDIS          1.000000
WCSTools        0.300813
Name: File_Name, dtype: float64

In [29]:
# Proportion of articles with aliases not giving recognizable credit

XML_all_no_rec = XML_results.loc[XML_results['rec_credit'] == "no"]
XML_no_rec_credit_proportion = XML_all_no_rec.groupby('Software_Package')['File_Name'].nunique()/XML_results.groupby('Software_Package')['File_Name'].nunique()
# XML_no_rec_credit_proportion.to_csv("XML_no_rec_credit_proportion_082019.csv")
XML_no_rec_credit_proportion

Software_Package
AstroBlend      1.000000
Astropy         0.144981
RADMC-3D        0.738318
SAOImage DS9    0.483871
Spec2d          0.595395
Stingray        1.000000
TARDIS          0.500000
WCSTools        0.861789
Name: File_Name, dtype: float64

In [30]:
# Total number of unique papers without some form of recognizable credit

XML_all_no_rec = XML_results.loc[XML_results['rec_credit'] == "no"]
XML_no_rec_total = XML_all_no_rec.groupby('Software_Package')['File_Name'].nunique()
XML_no_rec_total.to_csv("XML_no_rec_total_082019.csv")
XML_no_rec_total

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """


Software_Package
AstroBlend        1
Astropy          78
RADMC-3D        158
SAOImage DS9    165
Spec2d          181
Stingray          2
TARDIS            2
WCSTools        106
Name: File_Name, dtype: int64

In [31]:
# Total number of unique papers with some form of recognizable credit

XML_all_yes_rec = XML_results.loc[XML_results['rec_credit'] == "yes"]
XML_yes_rec_total = XML_all_yes_rec.groupby('Software_Package')['File_Name'].nunique()
XML_yes_rec_total.to_csv("XML_yes_rec_total_082019.csv")
XML_yes_rec_total

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """


Software_Package
AstroBlend        1
Astropy         531
RADMC-3D        192
SAOImage DS9    244
Spec2d          227
Stingray          1
TARDIS            4
WCSTools         74
Name: File_Name, dtype: int64

In [32]:
# Total number of unique papers with software aliases in the references section

XML_all_refs = XML_results.loc[XML_results['ref'] == "yes"]
XML_ref_count = XML_all_refs.groupby('Software_Package')['File_Name'].nunique()
XML_ref_count.to_csv("XML_ref_count_082019.csv")
XML_ref_count

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """


Software_Package
Astropy         463
RADMC-3D        146
SAOImage DS9     57
Spec2d          142
Stingray          1
TARDIS            4
WCSTools         37
Name: File_Name, dtype: int64

In [ ]:
# How many unique aliases were used in the references for each package?

print (XML_all_refs.groupby('Software_Package')['Alias'].nunique())

In [ ]:
# Which aliases were used in the papers with aliases in references? Write results to csv

XML_ref_aliases = pd.DataFrame({'ref_count' : XML_all_refs.groupby(['Software_Package', 'Alias', 'Identifier'])['File_Name'].nunique()}).reset_index()
XML_ref_aliases.to_csv("XML_ref_aliases_082019.csv")
XML_ref_aliases

In [ ]:
# Proportion of articles containing software aliases with the software mentioned in an acknowledgement

XML_all_ack = XML_results.loc[XML_results['ack'] == 'yes']
XML_ack_proportion = XML_all_ack.groupby('Software_Package')['File_Name'].nunique()/XML_results.groupby('Software_Package')['File_Name'].nunique()
XML_ack_proportion

In [ ]:
# Total number of unique papers with software aliases in acknowledgements

XML_all_ack = XML_results.loc[XML_results['ack'] == "yes"]
XML_ack_count = XML_all_ack.groupby('Software_Package')['File_Name'].nunique()
XML_ack_count.to_csv("XML_ack_count_082019.csv")
XML_ack_count

In [ ]:
# Which aliases were used in the papers with aliases in acknowledgements? Write results to csv

XML_ack_aliases = pd.DataFrame({'ack_count' : XML_all_ack.groupby([ "Software_Package", "Alias", "Identifier"])['File_Name'].nunique()}).reset_index()
XML_ack_aliases.to_csv("XML_ack_aliases_082019.csv")
XML_ack_aliases

In [ ]:
# Proportion of articles containing software aliases with the software mentioned in a footnote

XML_all_fn = XML_results.loc[XML_results['fn'] == 'yes']
XML_fn_proportion = XML_all_fn.groupby('Software_Package')['File_Name'].nunique()/XML_results.groupby('Software_Package')['File_Name'].nunique()
XML_fn_proportion

In [ ]:
# Total number of unique papers with software aliases in footnotes

XML_all_fn = XML_results.loc[XML_results['fn'] == "yes"]
XML_fn_count = XML_all_fn.groupby('Software_Package')['File_Name'].nunique()
XML_fn_count.to_csv("XML_fn_count_082019.csv")
XML_fn_count

In [ ]:
# Which aliases were used in the papers with aliases in footnotes? Write results to csv

XML_fn_aliases = pd.DataFrame({'fn_count' : XML_all_fn.groupby( [ "Software_Package", "Alias", "Identifier"])['File_Name'].nunique()}).reset_index()
XML_fn_aliases.to_csv("XML_fn_aliases_082019.csv")
XML_fn_aliases

In [ ]:
# Did any articles have no reference, acknowledgement, or footnote?

no_credit = XML_results.loc[(XML_results['fn'] == 'no') & (XML_results['ref'] == 'no') & (XML_results['ack'] == 'no')]
no_credit = no_credit.loc[~(no_credit['File_Name'].isin(XML_all_fn['File_Name']))]
no_credit = no_credit.loc[~(no_credit['File_Name'].isin(XML_all_refs['File_Name']))]
no_credit = no_credit.loc[~(no_credit['File_Name'].isin(XML_all_ack['File_Name']))]
no_credit.groupby('Software_Package')['File_Name'].nunique()